In [1]:
using Plots
using Statistics
using Surrogates
using CSV
using DataFrames
using Distributions
using Flux
import JSON
using MLUtils
using Plots
using StatsBase
using StatsPlots
using Tables

include("../../scan_hyperparameters_withcellnum_functions.jl")

main (generic function with 1 method)

### Loading data and making cuts for proper scaling

In [2]:
target_directory = "../../data/full_with_cellnumber/"

println("Formatting data...")
x_raw_df, y_df, cellnumber_df = getrawdata_withcellnum(target_directory)

# how to process cellnumber, for now let's make it another dvar
x_raw_origcn_df = hcat(x_raw_df, cellnumber_df)
x_raw_df = hcat(x_raw_df, cellnumber_df .% 2)

# cutting
#= println("Cutting Transmission to 60-100 percent...")
lower::Float32 = 60
upper::Float32 = 120
x_raw_df, y_df = applycut(x_raw_df, y_df, "OBJ1", lower, upper; with_numcells=true) =#

# decorrelating
println("Decorrelating...")
x_df = decorrelatedvars(x_raw_df; with_numcells=true)

x_scaled_df, x_scalers = minmaxscaledf(x_df);
y_scaled_df, y_scalers = minmaxscaledf(y_df);

println("Done!")

Formatting data...
Decorrelating...
Done!


In [3]:
y_scalers

Dict{String, MinMaxScaler} with 6 entries:
  "OBJ4" => MinMaxScaler(0.00582, 1.34901)
  "OBJ6" => MinMaxScaler(0.00697, 0.10912)
  "OBJ2" => MinMaxScaler(0.0542, 0.087)
  "OBJ3" => MinMaxScaler(35.05, 386.37)
  "OBJ5" => MinMaxScaler(0.00731, 0.10398)
  "OBJ1" => MinMaxScaler(1.53, 99.5)

A sanity check since Julia alphabetizes column names:

Recall that these are the original ranges of our DVARs

```
DVAR1: Bmax  [ 8.5, 12.0 ]
DVAR2: mX1  [ 5, 140 ]
DVAR3: mX2  [ 15, 160 ]
DVAR4: mY1  [ 1.005, 1.7 ]
DVAR5: mY2  [ 1.055, 1.85 ]
DVAR6: mtau1  [ 1, 500 ]
DVAR7: mtau2  [ 1, 500 ]
DVAR8: PhiY1  [ -89.95, -30 ]
DVAR9: PhiY2  [ -87.45, -25 ]
DVAR10: Phitau1  [ 1, 500 ]
DVAR11: Phitau2  [ 1, 500 ]
DVAR12: mY3ref  [ 1.105, 2.0 ]
DVAR13: PhiY3ref  [ -84.95, -20 ]
DVAR14: Eref  [ 0.055, 0.075 ]
```

but recall also that we decorrelate `DVAR3`, `DVAR5`, `DVAR9`, `DVAR12`, `DVAR13`. This means that their unscaled values will fall between 0 and 1.

In [4]:
x_scalers # looks good

Dict{String, MinMaxScaler} with 15 entries:
  "DVAR7"      => MinMaxScaler(1.00229, 499.997)
  "DVAR5"      => MinMaxScaler(-7.45177f-8, 0.999996)
  "DVAR12"     => MinMaxScaler(7.62605f-6, 0.999995)
  "DVAR14"     => MinMaxScaler(0.0550001, 0.075)
  "DVAR2"      => MinMaxScaler(5.00044, 139.998)
  "DVAR10"     => MinMaxScaler(1.00164, 499.998)
  "DVAR1"      => MinMaxScaler(8.50002, 12.0)
  "DVAR3"      => MinMaxScaler(1.07697f-5, 0.999996)
  "DVAR6"      => MinMaxScaler(1.00127, 499.998)
  "DVAR4"      => MinMaxScaler(1.005, 1.7)
  "DVAR8"      => MinMaxScaler(-89.95, -30.0006)
  "DVAR13"     => MinMaxScaler(2.18249f-5, 0.999996)
  "cellnumber" => MinMaxScaler(0.0, 1.0)
  "DVAR11"     => MinMaxScaler(1.00553, 499.997)
  "DVAR9"      => MinMaxScaler(3.582f-7, 0.999995)

In [5]:
println(names(x_df))
println(names(x_scaled_df))

["DVAR1", "DVAR2", "DVAR3", "DVAR4", "DVAR5", "DVAR6", "DVAR7", "DVAR8", "DVAR9", "DVAR10", "DVAR11", "DVAR12", "DVAR13", "DVAR14", "cellnumber"]
["DVAR1", "DVAR2", "DVAR3", "DVAR4", "DVAR5", "DVAR6", "DVAR7", "DVAR8", "DVAR9", "DVAR10", "DVAR11", "DVAR12", "DVAR13", "DVAR14", "cellnumber"]


In [6]:
@assert names(x_df) == names(x_scaled_df)

In [7]:
function decorrelate_single(data_to_transform, dynamic_lower_bound, strict_upper_bound; inverse=false)
    if inverse
        dynamic_lower_bound + data_to_transform*(strict_upper_bound - dynamic_lower_bound)
    else
        (data_to_transform - dynamic_lower_bound) / (strict_upper_bound - dynamic_lower_bound)
    end
end

decorrelate_single (generic function with 1 method)

In [8]:
function decorr(vec)
    # we are only decorrelating unscaled vectors, which means that these vectors have their DVARs in
    # quasi-numerical order DVAR1, ... DVAR14, cellnumber
    maxes = Dict(
        "DVAR3"=>160.,
        "DVAR5"=>1.85,
        "DVAR9"=>-25.,
        "DVAR12"=>2.,
        "DVAR13"=>-20.
    )
    etas = Dict(
        "DVAR3"=>10.,
        "DVAR5"=>0.05,
        "DVAR9"=>2.5,
        "DVAR12"=>0.05,
        "DVAR13"=>2.5
    )
    dvarmap = Dict(
        3=>2, # DVAR3prime ∼ Uniform(DVAR2prime + δ3, u3) 
        5=>4,
        9=>8,
        12=>5,
        13=>9
    )
    
    decorred_vec = []
    
    for i in 1:15
        dvar = (i == 15) ? "cellnumber" : "DVAR$i"
        # transform?
        if dvar ∈ keys(maxes)
            dependent_dvar = deps[dvar]
            append!(decorred_vec, getdvarprimesing(vec[i], etas[dvar] + vec[dvarmap[i]], maxes[dvar])) 
        else
            append!(decorred_vec, vec[i])
        end
    end
    decorred_vec
end

function recorr(vec)
    # we are only recorrelating scaled vectors, which means that these vectors have their DVARs in
    # quasi-alphabetical order DVAR1, DVAR10, ... DVAR2, ... DVAR9, cellnumber
    maxes = Dict(
        "DVAR3"=>160.,
        "DVAR5"=>1.85,
        "DVAR9"=>-25.,
        "DVAR12"=>2.,
        "DVAR13"=>-20.
    )
    etas = Dict(
        "DVAR3"=>10.,
        "DVAR5"=>0.05,
        "DVAR9"=>2.5,
        "DVAR12"=>0.05,
        "DVAR13"=>2.5
    )
    dvarmap = Dict(
        3=>2,
        5=>4,
        9=>8,
        12=>5,
        13=>9
    )
    
    recorred_vec = []
    
    for i in 1:14
        if i ∈ [3, 5, 9]
            append!(
                recorred_vec, decorrelate_single(vec[i], etas["DVAR$i"] + vec[dvarmap[i]], maxes["DVAR$i"]; inverse=true)
            )
        elseif i ∈ [12, 13]
            append!(
                recorred_vec,
                decorrelate_single(
                    vec[i], etas["DVAR$i"] + recorred_vec[dvarmap[i]], maxes["DVAR$i"]; inverse=true
                )
            )
        else
            append!(recorred_vec, vec[i])
        end
    end
    
    recorred_vec
end

function minmaxscale(vec; var="x", inverse=false)
    outvec = []
    
    transform_fun(scaler, x) = inverse ? inverse_transform(scaler, x) : transform(scaler, x)
    
    if var == "x"
        for i in 1:14
            scaler = x_scalers["DVAR$i"]
            append!(outvec, transform_fun(scaler, vec[i]))
        end
    elseif var == "y"
        for i in 1:6
            scaler = y_scalers["OBJ$i"]
            append!(outvec, transform_fun(scaler, vec[i]))
        end
    end
    
    return Tuple(outvec)
end

minmaxscale (generic function with 1 method)

### Loading Model

In [9]:
function model_id_to_nn(model_id, x_dim, y_dim)
    model_params = Dict()
    
    for substring in split(model_id, "_")
        if occursin("-", substring)
            continue
        elseif occursin("w=", substring)
            model_params["width"] = parse(Int, last(split(substring, "=")))
        elseif occursin("d=", substring)
            model_params["depth"] = parse(Int, last(split(substring, "=")))
        elseif occursin("activation=", substring)
            activation_function_string = last(split(substring, "="))
            model_params["activation"] = parseactivationfunctions([activation_function_string])[1]
        elseif occursin("dr=", substring)
            model_params["dropout_rate"] = parse(Float32, last(split(substring, "=")))
        end
    end
    
    neuralnetworkwithdropout(
        x_dim, y_dim, model_params["width"], model_params["depth"],
        model_params["dropout_rate"], model_params["activation"]
    )
end

model_id_to_nn (generic function with 1 method)

In [10]:
model_id = "2023-06-16_19-27-42_w=100_d=6_activation=sigmoid_bs=1024_lr=0.001_dr=0.0_1"
model_state = JLD2.load("../../models/$model_id.jld2", "model_state");
m = model_id_to_nn(model_id, size(x_df)[2], size(y_df)[2]);
Flux.loadmodel!(m, model_state);

#@load "../../models/2023-06-08_18-19-09_w=100_d=5_activation=sigmoid_bs=1024_lr=0.001_dr=0.0_1.jld2" m; old way

In [11]:
function predict(x_vec)
    m([x_vec...])'
end

predict (generic function with 1 method)

In [12]:
# do i understand how the model predicts for a given x_vec?
println(values(first(x_scaled_df)))

println("Prediction:")
println(predict(values(first(x_scaled_df))))
println("True:")
println(first(y_scaled_df))

(0.28919423f0, -0.3137467f0, 0.1475386862263235, 0.6688788f0, 0.5534791401182384, -0.7847122f0, -0.8160064f0, -0.45144355f0, 0.33135014022087317, -0.44775814f0, 0.4049362f0, 0.17109923841804942, 0.3576814045090202, -0.43408716f0, -1.0f0)
Prediction:


┌ Warning: Layer with Float32 parameters got Float64 input.
│   The input will be converted, but any earlier layers may be very slow.
│   layer = Dense(15 => 100, #69)  # 1_600 parameters
│   summary(x) = "15-element Vector{Float64}"
└ @ Flux ~/.julia/packages/Flux/EHgZm/src/layers/stateless.jl:60


Float32[0.8280521 -0.47372794 -0.63120365 -0.8962653 0.23376288 -0.08396596]
True:
DataFrameRow
 Row │ OBJ1      OBJ2  OBJ3       OBJ4       OBJ5      OBJ6       
     │ Any       Any   Any        Any        Any       Any        
─────┼────────────────────────────────────────────────────────────
   1 │ 0.826478  -0.5  -0.634009  -0.894922  0.304231  -0.0885951


### Functionality to predict cellnumber

In [13]:
ENV["PYTHON"] = ""                                                                                                                                   
using Pkg
Pkg.build("PyCall")
using PyCall

# Julia uses a Python that is installed to a separte location than my current anaconda3 installation.
# the following lines make sure that the correct Python packages are installed to read in the supplemental
# Python script that computes cell number

# using Conda
# Conda.add("scipy")
# Conda.add("matplotlib")

py"""
import sys
sys.path.insert(0, "./../../compute_cellnumber/")
"""

compute_cellnumber = pyimport("compute_cellnumber")["compute_cellnumber"]

    Building Conda ─→ `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/915ebe6f0e7302693bdd8eac985797dba1d25662/build.log`
    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/62f417f6ad727987c755549e9cd88c46578da562/build.log`


PyObject <function compute_cellnumber at 0x1df27e290>

In [14]:
# compute_cellnumber function uses different row keys
# this is a nice sanity check to make sure we're passing in the data through the correct slots
dvar_to_param = Dict(
    "DVAR1" => "Bmax",
    "DVAR2" => "mX1",
    "DVAR3" => "mX2",
    "DVAR4" => "mY1",
    "DVAR5" => "mY2",
    "DVAR6" => "mtau1",
    "DVAR7" => "mtau2",
    "DVAR8" => "PhiY1",
    "DVAR9" => "PhiY2",
    "DVAR10" => "Phitau1",
    "DVAR11" => "Phitau2",
    "DVAR12" => "mY3ref",
    "DVAR13" => "PhiY3ref",
    "DVAR14" => "Eref",
)

function jl_row_to_py_row(jl_row)
    pyrow = Dict()
    
    for i in 1:14
        pyrow[dvar_to_param["DVAR$i"]] = jl_row["DVAR$i"]
    end
    
    pyrow
end

jl_row_to_py_row (generic function with 1 method)

In [15]:
values(x_raw_df[10, :])

(11.181179f0, 132.81548f0, 158.4179f0, 1.6692148f0, 1.7696925f0, 475.04153f0, 402.1113f0, -60.956318f0, -48.767296f0, 410.49042f0, 20.040588f0, 1.8951204f0, -40.988632f0, 0.070214f0, 1)

In [16]:
compute_cellnumber(jl_row_to_py_row(x_raw_df[10, :]))

63

In [17]:
#= sanity check
cns_pred = []
cns_true = []

problematic_idx = []

for i in 1:size(x_raw_df)[1]
    
    if y_df[i, "OBJ1"] < 50
        continue
    else

        try
            cn = compute_cellnumber(jl_row_to_py_row(x_raw_df[i, :])) % 2
            append!(cns_pred, cn)
            append!(cns_true, x_raw_df[:, 15][i])
        catch
            append!(problematic_idx, i)
            continue
        end 
    end
end

histogram(cns_pred .- cns_true) =#

### Final optimization
- Maximize transmission (< 95%)
- Output energy should be 70 KeV (0.070 MeV)
- Minimize RFQ length
- Minimize longitudinal emittance
- Minimize transverse emittance

In [18]:
# optimization initialization
# bounds of scaled DVARS
lower_bound = repeat([-1.], 14); upper_bound = repeat([1.], 14)

# smapling to instantiate acquisition function
n_samples = 1000
initial_samples = Surrogates.sample(n_samples, lower_bound, upper_bound, SobolSample());

In [19]:
function objective3(x_vec)
    x_vec_unscaled = recorr(minmaxscale(x_vec; var="x", inverse=true))
    x_vec_unscaled_dict = Dict("DVAR$i" => x_vec_unscaled[i] for i in 1:14)
    local cellnumber
    try
        cellnumber = compute_cellnumber(jl_row_to_py_row(x_vec_unscaled_dict))
    catch
        cellnumber = 0
    end
    y = predict((x_vec..., (cellnumber % 2)*2 - 1.))
    target_energy_scaled = transform(y_scalers["OBJ2"], 0.07)[1]
    [-y[1], (y[2] - target_energy_scaled)^2, y[3], y[4], y[5], y[6]]
end

objective3 (generic function with 1 method)

In [20]:
x3s = [sample for sample in initial_samples];
x3s = unique(x3s)
y3s = objective3.(x3s);

In [21]:
if length(x3s) != length(unique(x3s))
    print("Duplicate entries in x3")
end
if length(x3s) != length(y3s)
    print("Dimension mismatch between x3 and y")
end
radial_basis_ego_3 = RadialBasis(x3s, y3s, repeat([-1.], 14), repeat([1.], 14))

(::RadialBasis{Surrogates.var"#1#2", Int64, Vector{NTuple{14, Float64}}, Vector{Vector{Float64}}, Vector{Float64}, Vector{Float64}, Matrix{Float64}, Float64, Bool}) (generic function with 1 method)

In [22]:
pareto_set_3, pareto_front_3 = surrogate_optimize(
    objective3, SMB(), repeat([-1.], 14), repeat([1.], 14), radial_basis_ego_3, SobolSample();
    maxiters = 100, n_new_look = 100
);

In [23]:
obj1_3 = [inverse_transform(y_scalers["OBJ1"], -item[1])[1] for item in pareto_front_3];
dE_3 = [item[2] for item in pareto_front_3];
obj3_3 = [inverse_transform(y_scalers["OBJ3"], item[3])[1] for item in pareto_front_3];
obj4_3 = [inverse_transform(y_scalers["OBJ4"], item[4])[1] for item in pareto_front_3];
obj5_3 = [inverse_transform(y_scalers["OBJ5"], item[5])[1] for item in pareto_front_3];
obj6_3 = [inverse_transform(y_scalers["OBJ6"], item[6])[1] for item in pareto_front_3];

unscaled_pareto_front_3 = [obj1_3, dE_3, obj3_3, obj4_3, obj5_3, obj6_3];

In [24]:
emit_cutoff = 0.04
long_emit_mask = obj4_3 .< emit_cutoff; x_emit_mask = obj5_3 .< emit_cutoff; y_emit_mask = obj6_3 .< emit_cutoff;


transmission_cutoff = 90
high_transmission_mask = obj1_3 .> transmission_cutoff

sum(long_emit_mask .& x_emit_mask .& y_emit_mask .& high_transmission_mask)

5

In [25]:
candidates = [i for i in 1:length(obj1_3)][long_emit_mask .& x_emit_mask .& y_emit_mask .& high_transmission_mask]

5-element Vector{Int64}:
  20
 103
 297
 399
 514

In [26]:
for j in candidates
    println("candidate $j")
    # println(pareto_set_3[j])
    candidate_dvars = minmaxscale(pareto_set_3[j]; var="x", inverse=true)
    println(recorr(candidate_dvars))
    #= for i in 1:6
        # println("OBJ$i: ", unscaled_pareto_front_3[i][j])
    end =#
    local cellnumber
    try
        cellnumber = compute_cellnumber(jl_row_to_py_row(Dict("DVAR$i" => recorr(candidate_dvars)[i] for i in 1:14)))
    catch
        cellnumber = 0
    end
    println(minmaxscale(predict([pareto_set_3[j]..., (cellnumber % 2)*2 - 1]); var="y", inverse=true))
    println(" ")
end

candidate 20
Any[8.89308809582144, 133.80184844136238, 156.97865025867986, 1.0966278220294043, 1.765510609608762, 150.60280898213387, 252.9359292089939, -85.09079340845346, -36.52964158344486, 169.12064218521118, 278.2774059474468, 1.8340688125928477, -33.440215182659706, 0.05935550605499884]
(93.34453130741349, 0.06548147392463366, 181.52716563539616, 0.03907475192160348, 0.03488076614336233, 0.029135358963253033)
 
candidate 103
Any[9.016133998520672, 116.39981546998024, 128.13922440468656, 1.1644984319573268, 1.2647673688317604, 94.07584205269814, 145.73012402653694, -74.55281127244234, -53.35124360536949, 385.48249435424805, 42.42575964331627, 1.6433319988371824, -44.855218953155344, 0.062246116453025024]
(90.15184353505538, 0.06374983255797773, 180.21827097863024, 0.03893252469185171, 0.029357717525449356, 0.02387355129610569)
 
candidate 297
Any[10.118420210201293, 84.29833798110485, 131.41610172082352, 1.0772846981999464, 1.8291769826105195, 363.797188565135, 78.72649578750134, 

In [27]:
testvec = [8.913598815910518, 122.46416029334068, 138.35337080606638, 1.2527336771599948, 1.7169229509382598, 488.7900076806545, 4.413380831480026, -63.663563065230846, -38.80922526825233, 468.323744058609, 140.86112385988235, 1.8500028655174254, -21.576793326001052, 0.06533203836443136]
println(minmaxscale(predict([testvec..., -1.]); var="y", inverse=true))
println(minmaxscale(predict([testvec..., 0.]); var="y", inverse=true))
println(minmaxscale(predict([testvec..., 1.]); var="y", inverse=true))

(49.52150868250828, 0.08108371052660956, 235.92447451705812, 0.010485917866276395, 0.03213730564380746, 0.029083502237326364)
(49.52186780973625, 0.08108326086921824, 235.89670767363714, 0.010481114244669243, 0.03211844956206367, 0.029074016176624973)
(49.522101388421106, 0.08108290114330519, 235.87034383060018, 0.01047639068342221, 0.03210063927981155, 0.029065038515197372)


#### An example

Earlier, I found the Pareto-optimal point
```
Any[9.621106587350368, 86.20992493629456, 151.5276411426766, 1.1407480351626873, 1.5461247062188697, 75.0710859298706, 90.66532409191132, -89.4816192984581, -57.44439364510753, 238.80475223064423, 402.53845977783203, 1.9589797589290225, -39.38285545256423, 0.056406302202958614]
```
Which is an RFQ with 87 cells. I predicted that this point would have the following OBJs
```
(99.11168172696327, 0.059405120364069454, 174.88078733049184, 0.03906433648567923, 0.033061393769183954, 0.0327312887736337)
```
The true values from PARMTEQM:
```
(97.72, 0.06190, 177.80, 0.07492, 0.04185, 0.04140)
```
With 87 cells. YAY!

In [28]:
function plot_pairwise_scatter(vec1, vec2, vec1label, vec2label, indexes, winner_index, incl_xticks, incl_yticks)
    loser_indexes = [i for i in indexes if i != winner_index]
    
    
    scatter(
        [vec1[i] for i in loser_indexes], [vec2[i] for i in loser_indexes];
        xlabel=vec1label, ylabel=vec2label, legend=false, markersize=3, markerstrokewidth=0.5,
        xformatter = incl_xticks ? :auto : Returns(""), yformatter = incl_yticks ? :auto : Returns("") 
    )
    
    scatter!([vec1[winner_index]], [vec2[winner_index]]; markershape=:star5, markersize=10, markerstrokewidth=0.5)
end

plot_pairwise_scatter (generic function with 1 method)

In [ ]:
winner_index = 514

labels = [
    "Transmission [%]",
    "Scaled energy\nsquared error",
    "RFQ length [cm]",
    "Longitudinal emit.\n[MeV deg]",
    "x-emit. [cm mrad]",
    "y-emit. [cm mrad]"
]
indexes = [i for i in 1:length(pareto_front_3)]

layout = @layout([ ° _ _ _ _ ; ° ° _ _ _ ; ° ° ° _ _ ; ° ° ° ° _ ; ° ° ° ° ° ])

#plot(p1, p2, p3, layout = l)

plots_dict = Dict()

for i in 1:(length(labels) - 1)
    for j in 1:length(labels)
        if i < j
            p = plot_pairwise_scatter(
                unscaled_pareto_front_3[i], unscaled_pareto_front_3[j],
                j==length(labels) ? labels[i] : "", i==1 ? ("\n" * labels[j]) : "", indexes, winner_index,
                j==length(labels), i==1
            )
            if i == 1
                xflip!(true)
            end
            # display(p)
            merge!(plots_dict, Dict("$i$j"=>p))
        end
    end
end
    
pareto_plot = plot(
    [plots_dict["$i$j"] for j in 1:length(labels) for i in 1:length(labels) if i<j]...,
    layout = layout, widths=(1/4,1/4,1/4,1/4), size=(1200,900), dpi=500
)

In [38]:
savefig(pareto_plot,"../../img/pareto_plot.png")

"/Users/Josh/Desktop/research/rfq-nn/img/pareto_plot.png"